<a href="https://colab.research.google.com/github/moonlovist/ML_DLA/blob/main/Get_DLA_catalog_mock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install fitsio

In [ ]:
!tar -xvf '/content/drive/My Drive/ML_DLA/files.tar.gz'

In [ ]:
!cp -r '/content/files' '/content/drive/My Drive/ML_DLA/'

In [ ]:
from fitsio import FITS
import fitsio
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.io.fits as pyfits
import numpy as np
import os
from matplotlib.pyplot import rcParams
import matplotlib._color_data as mcd
import math
rcParams['figure.figsize'] = 10, 5
rcParams['lines.linewidth'] = 2
rcParams['axes.labelsize'] = 15
rcParams['legend.fontsize'] = 12
from astropy.table import Table
import h5py
import sys

file_DLA = fitsio.FITS('/content/drive/My Drive/ML_DLA/zcat_desi_drq_DLA_sup_203.fits')
file_quasar = fitsio.FITS('/content/drive/My Drive/ML_DLA/zcat_desi_drq.fits')

Targetid_quasar = file_quasar[1]['THING_ID'][:].tolist()
Targetid_DLA = file_DLA[1]['THING_ID'][:].tolist()
Z_quasar = file_quasar[1]['Z'][:].tolist()
Z_DLA = file_DLA[1]['Z'][:].tolist()
NHI_DLA = file_DLA[1]['NHI'][:].tolist()
Targetid_list = list(set(Targetid_quasar).intersection(set(Targetid_DLA)))


data = np.load('/content/drive/My Drive/ML_DLA/spectra.npy',allow_pickle=True).tolist()
print(data.keys())
Targetid_all = data['Targetid']
spectra_all = data['spectra']
noise_all = data['noise_IVAR']
Z_all = data['z']
SN_all = data['SN']
B_wavelength = data['B_wavalength']

SN_range = (SN_all<3)#&(SN_all>3)
redshift_range = 2<Z_all
spectra_all = spectra_all[redshift_range&SN_range]
Targetid_all = Targetid_all[redshift_range&SN_range]
Z_all = Z_all[redshift_range&SN_range]
Targetid_list = list(set(Targetid_all).intersection(set(Targetid_DLA)))

total = len(Targetid_list)
spectra_dla = []
noise_dla = []
Z_quasar = []
Z_dla = []
NHI_dla = []
SN_dla = []
class_dla = []
tileid_dla = []
for index,id_quasar in enumerate(Targetid_list):
    position_quasar = Targetid_all.tolist().index(id_quasar)
    position_DLA = Targetid_DLA.index(id_quasar)
    Z_quasar = Z_all[position_quasar]
    B_wave = B_wavelength[(B_wavelength<1346*(Z_all[index]+1))&(B_wavelength>900*(Z_all[index]+1))]
    B_wave_position = B_wavelength.tolist().index(B_wave[0])
    for num in range(B_wave.shape[0]-399):
        B_wave_dla = B_wave[num:400+num]
        spectra_dla.append(spectra_all[position_quasar][B_wave_position+num:B_wave_position+400+num])
        Z_dla.append(Z_DLA[position_DLA])
        NHI_dla.append(NHI_DLA[position_DLA])
        tileid_dla.append(Targetid_DLA[position_DLA])
        center_dlla = 1210*(Z_DLA[position_DLA]+1)
        number1 = B_wave_dla[B_wave_dla>center_dlla].shape[0]
        number2 = B_wave_dla[B_wave_dla<center_dlla].shape[0]
        if (number1 > 60)&(number2 > 60):
            class_dla.append(1)
        else:
            class_dla.append(0)
    print("\r", end="")
    print("Download progress: {}%: ".format(round(index*100/total,2)), "▋" * ((int(index*100/total))), end="")
    sys.stdout.flush()
    
NHI_range = (np.array(NHI_dla) > 22)#&(np.array(NHI_dla) > 20.3)
    
hduspectra = fits.HDUList([fits.PrimaryHDU(np.array(spectra_dla)[NHI_range]),fits.BinTableHDU.from_columns([fits.Column(name='tileid_dla', array=np.array(tileid_dla)[NHI_range], format='K')])])
hduid = fits.HDUList([fits.PrimaryHDU(np.array(class_dla)[NHI_range]),fits.BinTableHDU.from_columns([fits.Column(name='tileid_dla', array=np.array(tileid_dla)[NHI_range], format='K')])])
hduredshift = fits.HDUList([fits.PrimaryHDU(np.array(Z_dla)[NHI_range]),fits.BinTableHDU.from_columns([fits.Column(name='tileid_dla', array=np.array(tileid_dla)[NHI_range], format='K')])])
hduNHI = fits.HDUList([fits.PrimaryHDU(np.array(NHI_dla)[NHI_range]),fits.BinTableHDU.from_columns([fits.Column(name='tileid_dla', array=np.array(tileid_dla)[NHI_range], format='K')])])
hduspectra.writeto('/content/drive/My Drive/ML_DLA/files_withid/Classification_catalog_SN_inf_3_NHI_22_mock_4.7.01.fits.gz')
hduid.writeto('/content/drive/My Drive/ML_DLA/files_withid/Classification_id_SN_inf_3_NHI_22_mock_4.7.01.fits.gz')
hduredshift.writeto('/content/drive/My Drive/ML_DLA/files_withid/Classification_z_SN_inf_3_NHI_22_mock_4.7.01.fits.gz')
hduNHI.writeto('/content/drive/My Drive/ML_DLA/files_withid/Classification_NHI_SN_inf_3_NHI_22_mock_4.7.01.fits.gz')



dict_keys(['Targetid', 'spectra', 'noise_IVAR', 'B_wavalength', 'z', 'SN'])
Download progress: 99.99%:  ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋